<a href="https://colab.research.google.com/github/DhibarSumit10/Online-Retail-Customer-Segmentation/blob/main/Online_Retail_Customer_Segmentation_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Extraction/identification of major topics & themes discussed in news articles. </u></b>

## <b> Problem Description </b>

### In this project, your task is to identify major customer segments on a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## <b> Data Description </b>

### <b>Attribute Information: </b>

* ### InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* ### StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
* ### Description: Product (item) name. Nominal.
* ### Quantity: The quantities of each product (item) per transaction. Numeric.
* ### InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
* ### UnitPrice: Unit price. Numeric, Product price per unit in sterling.
* ### CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
* ### Country: Country name. Nominal, the name of the country where each customer resides.

## **Introduction**

<b>**Customer segmentation is the practice of dividing a company's customers into groups that reflect similarity among customers in each group based on their shared behaviour or other attributes. Customer segmentation has the potential to allow marketers to address each customer in the most effective way. Using the large amount of data available on customers (and potential customers), a customer segmentation analysis allows marketers to identify discrete groups of customers with a high degree of accuracy based on demographic, behavioral and other indicators.The goal of segmenting customers is to decide how to relate to customers in each segment in order to maximize the value of each customer to the business. The groups should be homogeneous within them and should also be heterogeneous to each other. The main goal is to identify customers that are most profitable and loyal and the ones who churned out, to prevent further loss of customers by redefining company policies. Having a large number of customers, each with different needs it is difficult to find which customer is most important for business and target them with an appropriate strategy.**<b>

**The content of the notebook includes :-**

* **Data Preparation**
* **Data Preprocessing**
* **Feature Engineering**
* **Exploratory Data Analysis**
* **Model Building**
* **Conclusion**

#**Data Preparation**

###**1. Importing The Libraries and The Dataset**

In [ ]:
# Importing Required Libraries for our analysis

import pandas as pd
import numpy as np

# For plots and visualizations

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mpl_toolkits import mplot3d
from yellowbrick.cluster import SilhouetteVisualizer

# To scale the data
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Different Clustering Algorithms
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

# To get optimal number of clusters in hierarchical clustering

from scipy.cluster.hierarchy import dendrogram, linkage

# Matrics to evaluate the clusters
from sklearn.metrics import silhouette_score, silhouette_samples

# To avoid unnecessary warnings, let's import warnings also
import warnings
warnings.filterwarnings('ignore')

###**2. Let's Mount The Drive and Import The Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Let's define url path of the dataset

path = '/content/drive/MyDrive/Colab Notebooks/Online Retail Customer Segmentation - SUMIT KUMAR DHIBAR/Online Retail.xlsx'

# Now let's import dataset to crate a dataframe

cust_df = pd.read_excel(path)

###**3. Understanding the Dataset**

In [ ]:
# Let's check the shape of the dataframe

print('Shape of the dataset is :', cust_df.shape)

* **In our dataset we have 541909 rows and 8 columns.**



In [ ]:
# Checking all the columns present in our dataset

cust_df.columns

In [ ]:
# Checking top 5 records in DataFrame

cust_df.head()

In [ ]:
# Checking bottom 5 records in DataFrame

cust_df.tail()

In [ ]:
# Let's check the datatypes and null values of dataset

cust_df.info()

* **As it can be see from above that CustomerId, Description columns are having null values.**

In [ ]:
# Let's check the statistical description of the dataset

cust_df.describe().transpose()

### <b>4. Data Description </b>

### <b>Attribute Information : </b>

*  **InvoiceNo:** Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
*  **StockCode:** Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
*  **Description:** Product (item) name. Nominal.
*  **Quantity:** The quantities of each product (item) per transaction. Numeric.
*  **InvoiceDate:** Invoice Date and time. Numeric, the day and time when each transaction was generated.
*  **UnitPrice:** Unit price. Numeric, Product price per unit in sterling.
*  **CustomerID:** Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
*  **Country:** Country name. Nominal, the name of the country where each customer resides.

#**Data Preprocessing**

###**1. Missing Value Analysis**

**Real world data often has a lot of missing values. The cause of missing values can be failure to record data. The handling of missing data is very important during the preprocessing of the dataset as many of the machine learnng algorithms do not support missing values.**

In [ ]:
# Let's check the total missing data

cust_df.isnull().sum()

* **It can be seen from above that there are missing data in our dataset.**

In [ ]:
# Checking for missing values in our dataset with the help of heatmap.

plt.figure(figsize = (12,6))
sns.heatmap(cust_df.isnull(), cmap = 'YlGnBu')
plt.title('Heatmap for Missing Values in every column', weight = 'bold', fontsize = 14)
plt.show();

In [ ]:
# Percentage of missing data per category

Total_missing_data = cust_df.isnull().sum().sort_values(ascending = False)
Total_percentage = (cust_df.isnull().sum() / cust_df.isnull().count()).sort_values(ascending = False)*100
missing_data_concat = pd.concat([Total_missing_data, round((Total_percentage),2)], axis = 1, keys = ["Total No of Missing Values", "Percentage of Missing Values"])
missing_data = missing_data_concat[missing_data_concat['Total No of Missing Values']>0]
missing_data

* **It is clear from above that CustomerID has the highest percentage of missing values followed by Description column.**

In [ ]:
# Let's visualize the percentage of missing values

plt.figure(figsize = (13,8))
sns.set(style = 'darkgrid')
eval = sns.barplot(x = missing_data.index, y = missing_data['Percentage of Missing Values'], data = missing_data)
plt.title('Percentage of Missing Data', weight = 'bold')
plt.xlabel('Features', fontsize = 14, weight = 'bold')
plt.ylabel('Percentage of Missing Values', fontsize = 14, weight = 'bold')

for i in eval.patches:
   eval.annotate(f'{i.get_height()}', (i.get_x() + i.get_width() / 2., i.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

plt.show();

**As we can see from above 'CustomerID' has the highest percentage of missing values followed by 'Description' column - these features are having missing values. We will treat them as we go ahead in our analysis.**

In [ ]:
# Total Number of Null Values

print("Total number of null values in the data set : ", cust_df.isna().sum().sum())

# Let's count all the rows which contain missing values

count = 0
for i in cust_df.isna().sum(axis = 1):
  if i>0:
    count = count + 1
print('Total number of rows with missing values is ', count)
print(f'Percentage of rows which are having missing values in the entire dataset : {round((count/len(cust_df.index))*100, 2)}%')

**As it is not possible to impute CustomerID column with some value, we will not be able to form any clusters with those missing CustomerID. We will drop the missing values from the dataset in our further analysis.**

### **2. Handling Missing Data**

In [ ]:
# Let's first check the total number of unique values in every feature of the dataset

for i in cust_df.columns.tolist():
  print(f"Total number of unique values in '{i}' is : ", cust_df[i].nunique())

In [ ]:
# Let's drop the rows having null values

cust_df.dropna(subset = ['CustomerID'], inplace = True)


In [ ]:
# Let's check the shape of the dataset after removing all the rows having null values

print('Shape of the refreshed dataset is :', cust_df.shape)

In [ ]:
# Let's check the total missing data, if there is any

cust_df.isnull().sum()

**Hurray! As it can be seen from above that we have successfully handled all the missing values in the dataset.**

###**3. Handling Duplicate Data**

**It is important to remove duplicate records from the dataset, duplicate records may come from wrong collection of the data, it will add extra weight to our dataset and also increase time of the training.**

In [ ]:
# Let's check if any duplicate is present

print('Total number of duplicate rows :', cust_df.duplicated().sum())

In [ ]:
# Duplicate record gives extra weight to the dataset, it is better to remove them

cust_df.drop_duplicates(inplace = True)

# Let's check the shape of the dataset after removing all the duplicate values

print('Shape of the dataset after removing duplicate is :', cust_df.shape)

###**Let's change the datatype of the columns** 

In [ ]:
# Let's change the datatype of 'Quantity', 'UnitPrice' and 'CustomerID' column to the Integer

cust_df['Quantity'] = cust_df['Quantity'].astype(int)

cust_df['UnitPrice'] = cust_df['UnitPrice'].astype(int)

cust_df['CustomerID'] = cust_df['CustomerID'].astype(int)

###**Let's check why 'Invoice No' column is having object datatype.**

In [ ]:
# Let's workout on the object column

print('Value count for feature : InvoiceNo')
print('\n')
print(cust_df['InvoiceNo'].value_counts())

In [ ]:
# Let's check 'Invoice No' which contains letter 'C'

cust_df['InvoiceNo'] = cust_df['InvoiceNo'].astype(str)

cust_df[cust_df['InvoiceNo'].str.contains('C')]

**As it is clear from the above observation that 'InvoiceNo' column is having 'C' at begining , which indicates 'Cancellation'. We will remove those cancelled 'InvoiceNo' as we go ahead in our analysis.**

In [ ]:
# Let's drop those records from 'InvoiceNo' column, which contains 'C' in the begining

cust_df = cust_df[~ cust_df['InvoiceNo'].str.contains('C')]

# Let's check the shape of the dataset after removing all the Cancellation rows

print('Shape of the dataset after removing Cancellation is :', cust_df.shape)

In [ ]:
# Let's change the datatype of 'InvoiceNo' column to the Integer

cust_df['InvoiceNo'] = pd.to_numeric(cust_df['InvoiceNo'])

cust_df['InvoiceNo'] = cust_df['InvoiceNo'].astype(int)

In [ ]:
# Let's check the info of the dataset after doing operations on the datatypes

cust_df.info()

**Yes! so we have converted 'InvoiceNo', 'Quantity', UnitPrice', 'CustomerID' columns to integer datatype.**

#**Feature Engineering**

In [ ]:
# Creating new features from the 'InvoiceDate' column

cust_df['year'] = cust_df['InvoiceDate'].apply(lambda x: x.year)
cust_df['month'] = cust_df['InvoiceDate'].apply(lambda x: x.month_name())
cust_df['day'] = cust_df['InvoiceDate'].apply(lambda x: x.day_name())
cust_df['hour'] = cust_df['InvoiceDate'].apply(lambda x: x.hour)

In [ ]:
# Creating a new feature 'Total Amount' from 'unit Price' and 'Quantity' column
# 'Total Amount' = 'Unit Price' * 'Quantity'

cust_df['Total Amount'] = cust_df['Quantity'] * cust_df['UnitPrice']

In [ ]:
# Creating a new feature 'Time type' based on hours to define whether its 'morning' , 'Afternoon' or 'Evening'

def time_type(time):
  if (time > 5 and time < 12):
    return 'Morning'
  elif(time > 11 and time < 18):
    return 'Afternoon'
  else:
    return 'Evening'  

In [ ]:
cust_df['Time_type'] = cust_df['hour'].apply(time_type)

In [ ]:
# Let's check the dataset

cust_df.head()

**Yes! we have successfully created 6 new columns ['year', 'month', 'day', 'hour', 'Total Amount', 'Time Type'] from the dataset.**

#**Exploratory Data Analysis**

**Why do we perform EDA ?**

▶  An EDA is a thorough examination meant to uncover the underlying structure of a data set and is important for a company because it exposes trends, patterns and relationships that are not readily apparent. 

##**Univariate analysis**

###**Let's plot top 10 values based on frequency of each column**

In [ ]:
# First thing first - Let's define our target columns required to be analysed

column = ['InvoiceNo', 'StockCode', 'Quantity', 'Description', 'UnitPrice', 'CustomerID', 'Country', 'year', 'month', 'day', 'hour',
          'Total Amount', 'Time_type']

# Plotting countplots for different columns of our dataset

sns.set_style('darkgrid')
plt.figure(figsize = (18,30), dpi = 90)
for index, col in enumerate (column):
  plt.subplot(7,2,index+1)
  counts = cust_df[col].value_counts().reset_index().head(10)
  counts.rename(columns = {'index' : col, col : 'count'}, inplace = True)
  eval = sns.barplot(x= col, y = 'count', data = counts)
  plt.xticks(rotation = 20, ha = 'right')
  plt.title(f'Countplot of {col.title()}', weight = 'bold')
  plt.tight_layout()
  for i in eval.patches:
    eval.annotate(f'{i.get_height()}', (i.get_x() + i.get_width() / 2., i.get_height()), ha = 'center', xytext = (0, 10), textcoords = 'offset points')
plt.show();

###**Let's plot bottom 10 values based on frequency of each column**

In [ ]:
# First thing first - Let's define our target columns required to be analysed

b_column = ['StockCode' ,'Description','Quantity', 'CustomerID', 'Country']

# Plotting countplots for different columns of our dataset

sns.set_style('darkgrid')
plt.figure(figsize = (15,15), dpi = 90)
for index, col in enumerate (b_column):
  plt.subplot(3,2,index+1)
  counts = cust_df[col].value_counts().reset_index().tail(10)
  counts.rename(columns = {'index' : col, col : 'count'}, inplace = True)
  eval =  sns.barplot(x = col, y = 'count', data = counts)
  plt.xticks(rotation = 20, ha = 'right')
  plt.title (f'Countplot of {col.title()}', weight = 'bold')
  plt.tight_layout()
  for i in eval.patches:
     eval.annotate(f'{i.get_height()}', (i.get_x() + i.get_width() / 2., i.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')
plt.show()

###**Observations drawn from the above Univariate Analysis are as follows:**

* **'WHITE HANGING HEART T-LIGHT HOLDER' (Stock Code - 85123A ), 'REGENCY CAKESTAND 3 TIER' (Stock Code - 22423) are the top 2 most ordered products.**
* **Most customers are from 'United Kingdom' also considerable number of customers are also from 'Germany' , 'France', 'Eire' and 'Spain'. Whereas 'Saudi Arabia', 'Bahrain', 'Czech Republic', 'Brazil' and 'Lithuania' has least number of customers.**
* **Most of the customer have purchased items in the month of 'October', 'November', 'December' the reason may be most of the festivals are in these months. Less number of customers have purchased the items in the month of 'January', 'February', 'April'.**
* **There are no orders placed on 'Saturdays', the reason maybe all retail shop stay closed on this day.**
* **Most of the customers have purchased the items in Afternoon, moderate number of customer have purchased the items in Morning and least number of customers in Evening.**

###**Let's analyse Distributions of 'Total Amount' , 'Quantity' and 'Unit Price' columns.**

**Things to keep in mind :** 

* **Positively skewed (or right-skewed) distribution is a type of distribution in which most values are clustered around the left tail of the distribution while the right tail of the distribution is longer, which means mean>median>mode**


* **Negatively skewed (also known as left-skewed) distribution is a type of distribution in which more values are concentrated on the right side (tail) of the distribution graph while the left tail of the distribution graph is longer,which means mean<median<mode**

* **For symmetric graph mean=median=mode**

In [ ]:
# Let's plot the description of each of the columns

dis_col = ['Total Amount', 'Quantity', 'UnitPrice']

for i in dis_col:
  print('\n')
  print(f'Statistical Description of the feature : {i}')
  print('--'*25)
  print(cust_df[i].describe([0.75, 0.90, 0.95, 0.99]))

**As it can be seen from above that only 1 % of values in all 3 columns has the maximum price or Quantity range, it can be said that the distribution plots will be very much skewed.**

In [ ]:
# Visualizating the distributions of different columns

plt.figure(figsize = (20,5))
for n,col in enumerate (dis_col) :
  plt.subplot(1,3,n+1)
  sns.distplot(cust_df[col])
  plt.title(col.title(), weight = 'bold')
  plt.tight_layout()

###**Observations drawn from the above Distribution plot analysis are as follows:**

* **All the plots shows very skewed( Positively skewed ) distribution because most of the values are clustered around the left side of the distribution while the right tail of the ditribution are longer, which means mean > median > mode.**

**Why we use Log Transformation ?**

▶  When our original continuous data does not follow the bell curve, we can use log transformation on this data to make it as 'normal' as possible, so the analysis result from the data becomes more valid.

In [ ]:
# Let's apply log transformation on skewed clumns

for column in dis_col :
  fig, axes = plt.subplots(1, 2, figsize= (16,5) )
  sns.distplot(cust_df[column], ax = axes[0], color = 'blue').set(title = 'Before')
  try :
     sns.distplot(np.log(cust_df[column]), ax = axes[1], color = 'blue').set(title = 'After')
  except :   
     sns.distplot(np.log1p(cust_df[column]), ax = axes[1], color = 'blue').set(title = 'After')
  plt.suptitle(f'{column.title()}',weight='bold')
  plt.tight_layout()
  fig.show()

**It can be seen from above that after applying log transformation the distribution plot looks comparatively better than being skewed.**

##**Bivariate analysis**

###**1. Let's analyse top 5 countries with the most customer.**

In [ ]:
# First let's see the dataframe of top 5 Countries with most customers

country_cust = cust_df.groupby('Country')['CustomerID'].nunique().reset_index().sort_values('CustomerID', ascending = False)
country_cust.rename(columns = {'CustomerID' : 'Customer_count'}, inplace = True)
country_cust.head()

In [ ]:
# Let's plot a barplot to visualize the top 5 countries with most number of customer

plt.figure(figsize = (15,5))
sns.barplot(x = 'Country', y = 'Customer_count', data = country_cust.head(5), edgecolor = 'blue')
plt.title('Countries with most number of customers', weight = 'bold', fontsize = 15)
plt.show();

**It is clear from above that 'United Kingdom' has most number of customers than any other countries.**

###**2. Let's analyse top 5 countries with the most orders placed.**

In [ ]:
# First let's see the dataframe of top 5 Countries with most orders placed

country_ord = cust_df.groupby('Country')['InvoiceNo'].nunique().reset_index().sort_values('InvoiceNo', ascending = False)
country_ord.rename(columns = {'InvoiceNo' : 'Orders'}, inplace = True)
country_ord.head()

In [ ]:
# Let's plot a bar plot to visualize the top 5 countries with most orders placed

plt.figure(figsize = (15,5))
sns.barplot(x = 'Country', y = 'Orders', data = country_ord.head(5), edgecolor = 'blue')
plt.title ('Countries with highest orders' , weight = 'bold', fontsize = 15)
plt.show();

**'United kingdom' here also topped with most order placed compared to other countries.**

###**3. Let's analyse top 5 countries with orders in mass quantity.**

In [ ]:
# First let's see the dataframe of top 5 countries with orders in mass quantity

country_quant= cust_df.groupby('Country')['Quantity'].mean().reset_index().sort_values('Quantity', ascending = False)
country_quant.rename(columns = {'Quantity' : 'Avg_quant'}, inplace = True)
country_quant.head()

In [ ]:
# Let's plot a bar plot to visualize the top 5 countries with most orders placed

plt.figure(figsize = (15,5))
sns.barplot(x = 'Country', y = 'Avg_quant', data = country_quant.head(5), edgecolor = 'blue')
plt.title ('Countries with mass quantity orders placed' , weight = 'bold', fontsize = 15)
plt.show();

**It is clear from above that orders with mass quantity placed by the customer from Netherlands.**

###**4. Let's analyse top 5 items with most purchased according to quantity.**

In [ ]:
# First let's see the dataframe of top 5 items with most purchesed according to quantity

desc_quant = cust_df.groupby('Description')['Quantity'].sum().reset_index().sort_values('Quantity', ascending = False)
desc_quant.head()

In [ ]:
# Let's plot a bar plot to visualize the top 5 items with most purchesed according to quantity

plt.figure(figsize = (15,5))
sns.barplot(x = 'Description', y = 'Quantity', data = desc_quant.head(5), edgecolor = 'blue')
plt.title('Items with most orders', weight = 'bold', fontsize = 15)
plt.xticks(rotation = 20, ha = 'right')
plt.show();

**From above plot it can be stated that 'PAPER CRAFT , LITTLE BIRDIE' , 'MEDIUM CERAMIC TOP STORAGE JAR'  these are the top 2 items with most purchased in quantity.**		

###**5. Let's analyse top 5 items with highest total amount.**

In [ ]:
# First let's see the dataframe of top 5 items with highest total amount.

desc_amount = cust_df.groupby('Description')['Total Amount'].sum().reset_index().sort_values('Total Amount', ascending = False)
desc_amount.head()

In [ ]:
# Let's plot a bar plot to visualize the top 5 items with highest total amount.

plt.figure(figsize = (15,5))
sns.barplot(x = 'Description', y = 'Total Amount', data = desc_amount.head(5), edgecolor = 'blue')
plt.title('Items with highest revenue', weight  = 'bold', fontsize = 15)
plt.xticks(rotation = 20, ha = 'right')
plt.show();

**It can be stated that 'PAPER CRAFT , LITTLE BIRDIE' product has made highest revenue.**	

###**6. Let's analyse top 5 items purchased by most customer.**

In [ ]:
# First let's see the dataframe of top 5 items purchased by most customer.

item_customer = cust_df.groupby('Description')['CustomerID'].nunique().reset_index().sort_values('CustomerID', ascending = False)
item_customer.rename(columns = {'CustomerID' : 'Customer_count'}, inplace = True)
item_customer.head()

In [ ]:
# Let's plot a bar plot to visualize the top 5 items purchased by most customer.

plt.figure(figsize = (15,5))
sns.barplot(x = 'Description', y = 'Customer_count', data = item_customer.head(5), edgecolor = 'blue')
plt.title('Items with most customers', weight = 'bold', fontsize = 15)
plt.xticks(rotation = 20, ha = 'right')
plt.show();

**Item 'REGENCY CAKESTAND 3 TIER'	is the choice of most of the customer.**

#**Model Building**

##**RFM MODEL**

**Before applying any clustering algorithms it is always necessary to determine various quantitative factors on which the algorithm will perform segmentation. Examples of these would be features such as total amount spend by the particular customer, how frequent the customer visit the shop or what is the last visit of the customer.**

###**What is RFM ?**

**RFM Model which stands for Recency, Frequency and Monetary is one of such steps in which we determine the followings.**

* **Recency : How recently did the customer visit our website or how recently did the customer purchase.** 

* **Frequency : How often do they visit or how often do they purchase.**

* **Monetary : How much revenue we get from their visit or how much do they spend when they purchase.**

**However, these approach does not require machine learning algorithms as segmentation can be done manually.**

###**Why it is needed ?**

**RFM analysis is a marketing framework that is used to understand and analyze customer behaviour based on the above three factors Recency, Frequency and Monetary.**

**The RFM analysis will help the businesses to segment their customer base into different homogenous groups so that they  can engage with each group with different targeted marketing strategies.**

In [ ]:
# Let's first make a copy of the original dataset on which the model building will be done.

customer_df = cust_df.copy()

**Formula ➡**

* **Recency = Latest Date - Last Inovice Date.**
* **Frequency = count of invoice no. of transaction(s).** 
* **Monetary = Sum of Total Amount for each customer.**

In [ ]:
customer_df.columns

In [ ]:
import datetime as dt

# First let's set latest date 2011-12-10 as last invoice date was 2011-12-09. This is to calculate the number of days from recent purchase

Latest_Date  = dt.datetime(2011,12,10)

# Create RFM Modelling scors for each customer

rfm_df = customer_df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (Latest_Date - x.max()).days,
                                                'InvoiceNo': lambda x: len(x),
                                                'Total Amount': lambda x: x.sum()})

# Convert Invoice Date into type int

rfm_df['InvoiceDate'] = rfm_df['InvoiceDate'].astype(int)

# Rename column names to Recency, Frequency and Monetary

rfm_df.rename(columns={'InvoiceDate': 'Recency',
                       'InvoiceNo': 'Frequency',
                       'Total Amount': 'Monetary'}, inplace = True)

rfm_df.reset_index().head()

**Interpretation of above dataframe :**

1. Recency : How recent a customer made a purchase.
2. Frequency : How often a customer makes a purchase.
3. Monetary : How much money a customer spends in total.

In [ ]:
# Calculating R,F & M scores by splitting Recency, Frequency & Monetary based on quantities

rfm_df['R'] = pd.qcut(rfm_df['Recency'], q = 4, labels = [1,2,3,4]).astype(int)
rfm_df['F'] = pd.qcut(rfm_df['Frequency'], q = 4, labels = [4,3,2,1]).astype(int)
rfm_df['M'] = pd.qcut(rfm_df['Monetary'], q = 4, labels = [4,3,2,1]).astype(int)

# Finding the RFM group for each customer by combining the factors R, F & M

rfm_df['RFM_Group'] = 100*rfm_df['R'] + 10*rfm_df['F'] + rfm_df['M']

# Finding RFM score for each customer by adding the factors R, F & M

rfm_df['RFM_Score'] = rfm_df['R'] + rfm_df['F'] + rfm_df['M']

In [ ]:
# Let's see the dataframe afterr adding different columns

rfm_df.head()

**Interpretation of above dataframe :**
* If the R,F & M of any customer is 111, then we can say that the customers Recency is very good that means customer visited store very recently, Frequency is also very good for the resepective customer that means number of time the customer visited store is very high and also Monetary is very good for the customer which means the customer spend considerably high amount during his/her total visit at store.

* If the R,F & M of any customer is 444, then we can say that the customers Recency is very bad that means customer visited store while back, Frequency is also very bad for the resepective customer that means number of time the customer visited store is very less and also Monetary is very bad for the customer which means the customer spend very low amount during his/her total visit at store.

* If the R,F & M of any customer is 411, then we can say that the customer purchased long time ago but buys frequently and spends more

Like that we can come up with number of segments for all combinations of R,F & M. Lower the RFM score more valuable the customer is.

In [ ]:
# Let's understand the Descriptive Statistics of different columns in the above dataset

rfm_col = ['Recency', 'Frequency', 'Monetary']

for i in rfm_col:
  print('\n')
  print(f'Statistical Description of the feature : {i}')
  print('--'*25)
  print(rfm_df[i].describe([0.75, 0.90, 0.95, 0.99]))

**As it is clear from above that 'min' value of all the 3 columns are 0, which will give error in the transformations, we will treat them in further analysis.**

In [ ]:
# Handling zeros in the dataframe to avid error in the transformations

# Let's define a function to treat the zeros of the above dataframe

def zero_removal(num):
  if (num == 0):
    return 1
  else:
    return num

# Let's apply 'zero_removal' function to 'Recncy', 'Frequency' & 'Monetary' columns

rfm_df['Recency'] = [zero_removal(x) for x in rfm_df['Recency']]
rfm_df['Frequency'] = [zero_removal(x) for x in rfm_df['Frequency']]
rfm_df['Monetary'] = [zero_removal(x) for x in rfm_df['Monetary']]

In [ ]:
# Visualizing the distributions of different columns

plt.figure(figsize = (20,5))
for n,col in enumerate (rfm_col) :
  plt.subplot(1,3,n+1)
  sns.distplot(rfm_df[col])
  plt.title(col.title(), weight = 'bold')
  plt.tight_layout()

**It can be seen from above that 'Recency', 'Frequency' & 'Monetary' columns are very much rightly skewed (Positive skewed), we will treat them by applying log transformation.**

In [ ]:
# Applying Log Transformation on columns for smoothing the distribution

rfm_df['Recency_log'] = rfm_df['Recency'].apply(np.log)
rfm_df['Frequency_log'] = rfm_df['Frequency'].apply(np.log)
rfm_df['Monetary_log'] = rfm_df['Monetary'].apply(np.log)

In [ ]:
# Let's see the dataframe after adding new columns

rfm_df.head()

In [ ]:
# Checking Descriptive Statistics of the Dataframe

rfm_df.describe()

In [ ]:
# Visualizing the disatributions before and after the log transformation

target_column = ['Recency', 'Frequency', 'Monetary']

for column in target_column :
  fig, axes = plt.subplots(1, 2, figsize= (16,5) )
  sns.distplot(rfm_df[column], ax = axes[0], color = 'blue').set(title = 'Before')
  sns.distplot(np.log(rfm_df[column]), ax = axes[1], color = 'blue').set(title = 'After')
  plt.suptitle(f'{column.title()}',weight='bold')
  plt.tight_layout()
  fig.show()

**Observations**

* Earlier the distributions of Recency, Frequency and Monetary columns were positively skewed but after applying log transformation the distributions appear to be symmetrical and normally distributed.

* It will be more suitable to use the transformed features for better visualization of clusters.

###**Visualizing the correlations among RFM features.**

In [ ]:
rfm_coll = ["RFM_Group", "RFM_Score", "Recency_log", "Frequency_log", "Monetary_log"]

plt.figure(figsize = (10,6), dpi = 110)
sns.heatmap(abs(rfm_df[rfm_coll].corr()), annot = True, cmap = 'coolwarm')
plt.title('RFM Correlation Heatmap')
plt.show()

* **It is clear from above that 'Recency' is highly correlated with RFM_Group value , whereas Frequency and Monetary is moderately correlated with RFM_Group.**
* **Also RFM_Score is equally correlated with 'Recency',  'Frequency' and 'Monetary'.**

* **Note :**
        * RFM_Group : It is the concatination of R,F and M scores.
        * RFM_Score : It is the summation of R,F & M scores.

## **Feature Scaling**

In [ ]:
# Let's Define X Variable

X = rfm_df[['Recency_log', 'Frequency_log', 'Monetary_log']]

In [ ]:
# Let's Scale the variables with the help of StandardScaler method.

X = StandardScaler().fit_transform(X)

:* **Mean and Standard Deviation of values in the RFM features after scaling are 0 and 1 respectively.**

##**Clustering**

In [ ]:
# Saving the result in results variable

results = { }

# Let's define a function to remove the outliers

def treating_outliers(col):
  '''
      This above function takes a column and removes the outliers
      col : The respective column which is to be cleaned
  '''
  col_q1 = col.quantile(0.25)
  col_q3 = col.quantile(0.75)
  col_iqr = col_q3 - col_q1
  condition_1 = (col >= col_q1 - 1.5 * col_iqr)
  condition_2 = (col <= col_q3 + 1.5 * col_iqr)
  return col[condition_1 & condition_2]

# Now let's define a function for displaying the mean and median of Recency, Frequency and Monetary for each group of customers

def cluster_stats(dataset, segment):
  '''
    This above function returns the mean, median and count of Recency, Frequency and Monetary for each group of customers
  '''
  statistic = dataset[['Recency', 'Frequency', 'Monetary', segment]].groupby(segment).agg(['mean', 'median'])
  statistic['count'] = dataset[segment].value_counts()

# Let's store 0.25th and 0.75th quantile of Recency, Frequency & Monetary for each group of customers

result = dataset[['Recency', 'Frequency', 'Monetary', segment]].groupby(segment).agg({
                'Recency': lambda x : f'{int(x.quantile(0.25))} to {int(x.quantile(0.75))} days ago',
                'Frequency' : lambda x : f'Bought {int(x.quantile(0.25))} to {int(x.quantile(0.75))} times',
                'Monetary' :  lambda x : f'Spend around {int(x.quantile(0.25))} to {int(x.quantine(0.75))} pound Sterling',
                })

# Let's rename the columns
statistic.columns = [f"{i}  {j}" for i, j in statistic.columns]
result.columns = ['Last visited', 'Purchase frequency', 'Amount spent']

return statistic, result


# Now let's define function for plottting clusters for visualization

def plot_statistic_result(y, segment, title):
  '''
    This above function plots the clusters for the given data and returns the dataframes for the clusters with con